
# Flower Recognition CNN Keras



In [ ]:
import os
os.chdir('/kaggle/input/flowers-recognition/flowers') 
print(os.listdir())

## CONTENTS ::


## 1 ) Importing Various Modules.

## 2 ) Preparing the Data

## 3 ) Modeling

## 4 ) Evaluating the Model Performance

## 5 ) Visualizing Predictons on the Validation Set

In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras


## 1 ) Importing Various Modules.

In [ ]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
 
#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models
import tensorflow as tf

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import random as rn
# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
import glob
import os                   



## 2 ) Preparing the Data

## 2.1) Making the functions to get the training and validation set from the Images

In [ ]:

IMG_SIZE=150
FLOWER_DAISY_DIR='/kaggle/input/flowers-recognition/flowers/daisy'
FLOWER_SUNFLOWER_DIR='/kaggle/input/flowers-recognition/flowers/sunflower'
FLOWER_TULIP_DIR='/kaggle/input/flowers-recognition/flowers/tulip'
FLOWER_DANDI_DIR='/kaggle/input/flowers-recognition/flowers/dandelion'
FLOWER_ROSE_DIR='/kaggle/input/flowers-recognition/flowers/rose'


In [ ]:
# Create a dictionary to store counts for each directory
image_counts = {}

# Loop through the directories and count images
for directory, dir_name in zip([FLOWER_DAISY_DIR, FLOWER_SUNFLOWER_DIR, FLOWER_TULIP_DIR, FLOWER_DANDI_DIR, FLOWER_ROSE_DIR], ['Daisy', 'Sunflower', 'Tulip', 'Dandelion', 'Rose']):
    image_paths = glob.glob(os.path.join(directory, '*.jpg'))
    image_count = len(image_paths)
    image_counts[dir_name] = image_count

# Print the image counts for each directory
for dir_name, count in image_counts.items():
    print(f"Directory '{dir_name}' contains {count} images.")

In [ ]:

# Initialize empty lists for data and labels
X = []
y = []

In [ ]:

# Define a function to load and preprocess images for a specific category
def load_and_preprocess_category(category, directory):
    image_paths = glob.glob(os.path.join(directory, '*.jpg'))
    for image_path in image_paths:
        img = cv2.imread(image_path)
        if img is not None:
            img = cv2.resize(img, (128, 128))  # Resize the image
            img = img / 255.0  # Normalize pixel values
            X.append(img)
            y.append(category)

In [ ]:
# Call the function for each flower category
load_and_preprocess_category('Daisy', FLOWER_DAISY_DIR)
load_and_preprocess_category('Sunflower', FLOWER_SUNFLOWER_DIR)
load_and_preprocess_category('Tulip', FLOWER_TULIP_DIR)
load_and_preprocess_category('Dandelion', FLOWER_DANDI_DIR)

# Encode labels using LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Convert the lists to NumPy arrays
X = np.array(X)
Y = np.array(y_encoded)

In [ ]:
X

In [ ]:
Y

In [ ]:
print(len(X))

In [ ]:
print(len(Y))

In [ ]:

from sklearn.model_selection import train_test_split

# Split the data into training (70%) and temporary data (30%)
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.30, random_state=42)

# Split the temporary data into validation (15%) and testing (15%)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.50, random_state=42)

In [ ]:

# Define the CNN model
model = keras.Sequential([
    # Convolutional layer 1
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    
    # Convolutional layer 2
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Convolutional layer 3
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Flatten the output for dense layers
    layers.Flatten(),
    
    # Dense layer 1
    layers.Dense(128, activation='relu'),
    
    # Output layer with softmax activation for classification
    layers.Dense(5, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

In [ ]:
# Training & Evaluation

model.fit(X_train,Y_train, epochs=10 , batch_size=32 )

In [ ]:
score = model.evaluate (X_test,Y_test)
print("Test Score:",score[0])
print("Test Accuracy:",score[1])
